In [1]:
import psycopg2
import pandas as pd

In [18]:
def getData(table) :
    """
    DB에서 검색 단어가 포함된 데이터를 불러오고 dataFrame 생성 
    ...
    parameters
        table : 데이터가 저장되어 있는 DB 내 테이블 명 (str)
        clean_word : 형태소 단위로 정제된 단어 (str)
    ...
    returns
        df : DB에서 가져온 데이터가 담긴 데이터프레임 (dataFrame; shape[n,4])
    """
    #postgresql 접속
    conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()

    #데이터 가져오기
    command = "select distinct title, date, url, hashtag from " + table
    cur.execute(command)
    conn.commit()
    data = cur.fetchall()

    #DF화
    df = pd.DataFrame(data)
    df.columns = ['title', 'date', 'url', 'hashtag']

    #db 연결 종료
    conn.cursor().close()
    conn.close()

    return df

In [28]:
#insert문 만들기
def insertData(table, df) :
    """
    공용 DB에 내 데이터 insert
    ...
    parameters
        table : 데이터를 저장할 DB
    ...
    returns
        None
    """

    #postgresql 접속
    conn_string = "host='kedu.postgres.database.azure.com' dbname='postgres' user='keduadmin' password='1qaz2wsx!' port='5432'"
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()

    command = 'insert into kedu5.' + table + ' (title, date, url, hashtag) values (%s, %s, %s, %s);'
    
    #table에 데이터 넣기
    for i in range(len(df)) :
        cur.execute(command, (df.iloc[i][0], df.iloc[i][1], df.iloc[i][2], df.iloc[i][3]))
        conn.commit()

    #db 연결 종료
    conn.cursor().close()
    conn.close()

In [30]:

#postgresql 접속
conn_string = "host='kedu.postgres.database.azure.com' dbname='postgres' user='keduadmin' password='1qaz2wsx!' port='5432'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

command = """create table kedu5.view_interior (
	id serial,
	title varchar(110),
	date date,
	url varchar(300),
	hashtag varchar(500));"""

cur.execute(command)

#db 연결 종료
conn.cursor().close()
conn.close()

In [67]:
# set search_path = kedu5,public;

#postgresql 접속
conn_string = "host='kedu.postgres.database.azure.com' dbname='postgres' user='keduadmin' password='1qaz2wsx!' port='5432'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

cur.execute('select * from kedu5.view_interior limit(5)')
conn.commit()
cur.fetchall()

[(1,
  '하포스 빅와이드서랍 LED침대세트 9존독립매트리스 포함 슈퍼싱글 퀸, 303수납형(오크), 9존독립 화이트...',
  datetime.date(2022, 6, 1),
  'https://blog.naver.com/srqup54/222754929277',
  ''),
 (2,
  '얇은 이불로 바꾸셨나요?',
  datetime.date(2022, 5, 11),
  'https://cafe.naver.com/glove/7481259',
  ''),
 (3,
  '호텔침대 프레임 크라운구스',
  datetime.date(2022, 5, 1),
  'https://blog.naver.com/gkduddl88/222717240428',
  '#호텔침대 #침대프레임 #크라운구스'),
 (4,
  '슈퍼킹사이즈로 부부 침대 바꿨어요',
  datetime.date(2022, 4, 16),
  'https://cafe.naver.com/imsanbu/60479280',
  ''),
 (5,
  '허리쿠션으로 편하게! 오토노스 사무용 학생의자 추천',
  datetime.date(2022, 4, 5),
  'https://blog.naver.com/flythfk2/222692075760',
  '#오토노스 #오토노스 #밤밤쿠션 #사무용의자 #학생의자 #허리쿠션 #사무실의자 #책상의자 #의자등쿠션 #허리등받이쿠션')]

In [71]:
df = getData('view_interior')
print(df.shape)
df.head()

(13045, 4)


,title,date,url,hashtag
0,"하포스 빅와이드서랍 LED침대세트 9존독립매트리스 포함 슈퍼싱글 퀸, 303수납형(...",2022-06-01,https://blog.naver.com/srqup54/222754929277,
1,얇은 이불로 바꾸셨나요?,2022-05-11,https://cafe.naver.com/glove/7481259,
2,호텔침대 프레임 크라운구스,2022-05-01,https://blog.naver.com/gkduddl88/222717240428,#호텔침대 #침대프레임 #크라운구스
3,슈퍼킹사이즈로 부부 침대 바꿨어요,2022-04-16,https://cafe.naver.com/imsanbu/60479280,
4,허리쿠션으로 편하게! 오토노스 사무용 학생의자 추천,2022-04-05,https://blog.naver.com/flythfk2/222692075760,#오토노스 #오토노스 #밤밤쿠션 #사무용의자 #학생의자 #허리쿠션 #사무실의자 #책...


In [75]:
df['date'] = pd.to_datetime(df.date)
df.drop(df[df['date'] > '2022-05-31'].index, inplace=True)
df.shape

(11474, 4)

In [76]:
df.to_csv('view_interior.csv', encoding='utf-8-sig', index=False)

In [68]:
view_interior = pd.read_csv('view_interior.csv', encoding='utf-8')
view_interior.head()

,title,date,url,hashtag
0,"하포스 빅와이드서랍 LED침대세트 9존독립매트리스 포함 슈퍼싱글 퀸, 303수납형(...",2022-06-01,https://blog.naver.com/srqup54/222754929277,NaN
1,얇은 이불로 바꾸셨나요?,2022-05-11,https://cafe.naver.com/glove/7481259,NaN
2,호텔침대 프레임 크라운구스,2022-05-01,https://blog.naver.com/gkduddl88/222717240428,#호텔침대 #침대프레임 #크라운구스
3,슈퍼킹사이즈로 부부 침대 바꿨어요,2022-04-16,https://cafe.naver.com/imsanbu/60479280,NaN
4,허리쿠션으로 편하게! 오토노스 사무용 학생의자 추천,2022-04-05,https://blog.naver.com/flythfk2/222692075760,#오토노스 #오토노스 #밤밤쿠션 #사무용의자 #학생의자 #허리쿠션 #사무실의자 #책...


In [70]:
words = ['의자', '학생']
df = view_interior[view_interior['title'].map(lambda x: all(tt in x for tt in words))]
df

,title,date,url,hashtag
4,허리쿠션으로 편하게! 오토노스 사무용 학생의자 추천,2022-04-05,https://blog.naver.com/flythfk2/222692075760,#오토노스 #오토노스 #밤밤쿠션 #사무용의자 #학생의자 #허리쿠션 #사무실의자 #책...
104,초등학생책상의자무회전 인기상품 모음입니다.,2022-05-05,https://blog.daum.net/drnopain/6682,NaN
176,삼공체어 중고등학생의자 메쉬재질에 목받침까지 있어 정말 편해요.,2021-09-19,https://blog.naver.com/minkknim/222510816385,#삼공체어 #중고등학생의자 #메쉬재질의자 #학생의자 #목받침의자 #메쉬의자 #편한의...
208,"학생의자 로고스토리 학생 컴퓨터 사무용 의자 헤드형 AIRWH10, 화이트프레임 +...",2022-02-21,https://blog.daum.net/jgs76041588/125,NaN
229,퍼시스 서울대의자 CHNA4311WA 학생용의자 공부용의자 독서실의자 스터디카페의자...,2022-05-04,https://blog.naver.com/wqc969g/222719643864,NaN
...,...,...,...,...
11386,시디즈 학생의자 패브릭커버 베스트 상품 리스트,2022-05-23,https://blog.daum.net/cop-mi/2049,NaN
11435,시니프 로보 자세교정 학생의자,2022-04-20,https://blog.naver.com/okidarim/222706408619,NaN
11498,학생의자추천,2021-11-13,https://cafe.naver.com/wooseongapt/413576,NaN
11523,"다니카 스마트 포켓스프링 학생 사무용의자 헤드형 GMHD-901, 블랙 (hit한거...",2022-06-13,https://blog.naver.com/6xkbjb4j7ib32/222769451786,NaN
